In [1]:
from datasets import load_dataset

/Users/dsmilkov/code/lilac/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [71]:
import os

ds = load_dataset('Open-Orca/SlimOrca')['train']
print(os.path.basename(ds._data.blocks[0][0].path))
print(os.path.basename(ds._data.blocks[1][0].path))

slim_orca-train-00000-of-00002.arrow
slim_orca-train-00001-of-00002.arrow


In [72]:
def func(example):
  gpt = [x['value'] for x in example['conversations'] if x['from'] == 'gpt']
  system = [x['value'] for x in example['conversations'] if x['from'] == 'system']
  human = [x['value'] for x in example['conversations'] if x['from'] == 'human']
  res = {}
  res['gpt'] = gpt[0] if gpt else None
  res['system'] = system[0] if system else None
  res['human'] = human[0] if human else None
  res['human_and_gpt'] = res['human'] + '\n' + res['gpt']
  return res


ds2 = ds.map(func, remove_columns=['conversations'])

In [75]:
ds3 = ds2.map(lambda x: {'test': (x or '') + ''}, input_columns='system')


Map: 100%|██████████| 517982/517982 [00:36<00:00, 14365.22 examples/s]


In [79]:
print(ds3._data.path)
print(ds3._data.schema)

/Users/dsmilkov/.cache/huggingface/datasets/Open-Orca___slim_orca/default-8eee9bed99b72d92/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-584016660c217c6d.arrow
gpt: string
system: string
human: string
human_and_gpt: string
test: string
-- schema metadata --
huggingface: '{"info": {"features": {"gpt": {"dtype": "string", "_type": ' + 246


In [68]:
new_column = ["foo"] * len(ds)
ds3 = ds.add_column("new_column", new_column)
print(os.path.basename(ds3._data.blocks[0][0].path))
print(os.path.basename(ds3._data.blocks[1][0].path))
print(type(ds3._data.blocks[0][1]))
print(type(ds3._data.blocks[1][1]))

slim_orca-train-00000-of-00002.arrow
slim_orca-train-00001-of-00002.arrow
<class 'datasets.table.InMemoryTable'>
<class 'datasets.table.InMemoryTable'>


In [44]:
def func(example):
  gpt = [x['value'] for x in example['conversations'] if x['from'] == 'gpt']
  system = [x['value'] for x in example['conversations'] if x['from'] == 'system']
  human = [x['value'] for x in example['conversations'] if x['from'] == 'human']
  res = {}
  res['gpt'] = gpt[0] if gpt else None
  res['system'] = system[0] if system else None
  res['human'] = human[0] if human else None
  res['human_and_gpt'] = res['human'] + '\n' + res['gpt']
  return res


ds2 = ds.map(func, remove_columns=['conversations'])

In [46]:
ds3 = ds.map(func)

['conversations']


In [45]:
import duckdb

print(type(ds2._data))
print(ds2._data.path)
arrow_table = ds2._data.table
con = duckdb.connect()
con.execute(
  'SELECT gpt, COUNT(*) as c FROM arrow_table GROUP BY gpt ORDER BY c DESC LIMIT 10').fetchall()


<class 'datasets.table.MemoryMappedTable'>
/Users/dsmilkov/.cache/huggingface/datasets/Open-Orca___slim_orca/default-8eee9bed99b72d92/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-8dd92d211d6fd44c.arrow


[('Yes', 1787),
 ('No', 1536),
 ('False', 1137),
 ('Insufficient information to provide an answer.', 716),
 ("Can't answer", 475),
 ('No answer', 404),
 ('Not answerable.', 381),
 ('True', 377),
 ('1', 312),
 ('No.', 300)]

In [23]:
ds2.save_to_disk('~/orcaslim/')

Saving the dataset (4/4 shards): 100%|██████████| 517982/517982 [00:04<00:00, 123040.31 examples/s]


In [31]:
ds2._data.table.schema

space = 0


def starts_with_whitespace(ex):
  global space
  if ex[0].isspace():
    space += 1
    print(ex)
  return


space = 0
ds2.map(starts_with_whitespace, input_columns=['human'])
print(space)

Map:  89%|████████▊ | 459447/517982 [00:07<00:00, 60011.98 examples/s]

 Is The Invisible Man more prevalent in films than Picnic at Hanging Rock? Given the above question, please answer with reasoning first!


Map:  96%|█████████▌| 498467/517982 [00:08<00:00, 60086.82 examples/s]

 _carry signals that control sexual development and reproduction
Options:
- (A) sex hormones
- (B) electricity
- (C) nucleic acid
- (D) Most plants
- (E) animals
- (F) genes
- (G) dendrites
- (H) photons
--
Let's think first: Sex hormones are chemical messengers that control sexual development and reproduction. Chemical messengers carry signals from one nerve branch to another. Sex hormones carry signals that control sexual development and reproduction.... So the answer is [(A)]

What are a source of food for many fish species?
Options:
- (A) layers of fat
- (B) bird species
- (C) human species
- (D) keystone species
- (E) Organic compounds
- (F) barnacles
- (G) gastropod shells
- (H) species of deer
--
Let's think first: Sponges are a source of food for many species of fish. Sponges are also keystone species. Keystone species are a source of food for many fish species.... So the answer is [(D)]

Many plants respond to the days growing shorted by doing what?
Options:
- (A) colder and w

Map: 100%|██████████| 517982/517982 [00:08<00:00, 59684.17 examples/s]

2


In [1]:
import lilac as ll

/Users/dsmilkov/code/lilac/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ds = ll.get_dataset('local', 'orca-slim', project_dir='./data')

In [14]:
df = ds.to_pandas()
df['gpt.near_dup'] = df['gpt.near_dup'].apply(lambda x: x['cluster_id'])
df['human.near_dup'] = df['human.near_dup'].apply(lambda x: x['cluster_id'])


In [17]:
print(len(df))
df_dedup = df.drop_duplicates('human.near_dup', keep='first')

517982


In [18]:
len(df_dedup)

457719